In [ ]:
from multiprocessing import Pool
import glob
import json
from tqdm import tqdm
from scipy.spatial import distance


In [72]:
json_list = glob.glob("/home/alien_yhl/data/results_6w/"+"*_result.json")

def job(js):
    # json_list = glob.glob("/home/alien_yhl/downloads/results_json/english_23952/"+"*_result.json")


    #calculate euclidean distance
    dists = []
    failures = []
    special = []
    rdi = []
    
    with open(js, 'r') as f:
        data = json.load(f)
    alignment_results = data["alignment_results"]
    
    for i in alignment_results.keys():
        sample_mkpts = alignment_results[i]["matched_features"]["sample_mkpts"]
        template_mkpts = alignment_results[i]["matched_features"]["template_mkpts"]
        ratio = alignment_results[i]["matched_features"]["mkpts_ratio2template"]
        isGT = alignment_results[i]["isGroundTruthTemplate"]
        mkpts_conf = alignment_results[i]["matched_features"]["mkpts_conf"]
        
        aver_conf = sum(mkpts_conf) / len(mkpts_conf)
        if isGT == 1:
            mark += 1
        conf = alignment_results[i]["matched_features"]["mkpts_conf"]
        dist = 0; weighted_dist = 0;
        if len(sample_mkpts) != len(template_mkpts):
            print("length not same".format(data["id"]))
            
        for j in range(len(sample_mkpts)):
            
            dist += distance.euclidean(sample_mkpts[j], template_mkpts[j])
            weighted_dist += distance.euclidean(sample_mkpts[j], template_mkpts[j])*conf[j]
            dist = dist / (j+1); weighted_dist = weighted_dist / (j+1);

        dists.append(dist)
        # with open("/home/alien_yhl/workspace/img_align/logs/test2/"+data["id"]+"_"+i+".txt", 'w') as f:
        #     f.writelines(data["id"])
        #     print("Done {}".format(data["id"]))
        return (ratio, dist, isGT, data["id"], weighted_dist, aver_conf) ####MOST IMPORTANT!





In [84]:
with Pool(4) as p:
    # for js in tqdm(json_list):
    #     p.apply_async(job, (js, ), error_callback=lambda x: print(x))
    data_list = p.map(job, tqdm(json_list))
p.close()
p.join()



100%|██████████| 15460/15460 [01:04<00:00, 241.32it/s]


10

In [85]:
print(len(data_list))

[(0.6142578125, 0.025660736045354005, 1, 'c87d7405-a9ff-44f8-9060-ee4d4d8a583e', 0.024609692906407063, 0.9493769891500852), (0.658203125, 0.017879330004684016, 1, '779c5126-775b-4061-a97c-9f0712ce4355', 0.01725035730406898, 0.9515197147955173), (0.4515539305301645, 0.07758089678520676, 1, '75fcf1b3-ecec-40c2-ab61-053fd7fb0466', 0.03992376814195897, 0.9003738379671506), (0.4691516709511568, 0.11541596911239957, 1, 'c7c5c7e7-0968-42b9-89af-7e0b27c2ff1c', 0.06651089793252601, 0.9217845812235793), (0.44921875, 0.059298598600382786, 1, '34456c6d-e7b6-464c-85e1-3469f16c6c7c', 0.05637990922784703, 0.9313726517169372), (0.55078125, 0.08046646383290805, 1, '77282ac2-2d22-4fa3-83c0-08406d7b954c', 0.04608124856655713, 0.9332620511029629), (0.36988847583643125, 0.6589484904674696, 1, '9a5363a8-1e29-45c3-adab-1e8a921b89b0', 0.3717887951688266, 0.8973785722675036), (0.7081497797356828, 0.0034863133489635824, 1, '402a5091-bfcb-42b8-bfe4-0652a306a10b', 0.0017616565057786342, 0.951143660960635), (0.385